In [1]:
%load_ext dotenv
%dotenv

import os
print([k for k in os.environ.keys() if "GOOGLE" in k])

# unset google vertex api
# import os
# for k in [
#     "GOOGLE_GENAI_USE_VERTEXAI",
#     "GOOGLE_CLOUD_PROJECT",
#     "GOOGLE_CLOUD_LOCATION",
# ]:
#     os.environ.pop(k, None)
# print("Vertex mode:", os.getenv("GOOGLE_GENAI_USE_VERTEXAI"))

['GOOGLE_API_KEY']


In [2]:
# sample zinc dataset
from datasets.iterable_dataset import IterableDataset
from datasets import load_dataset

dataset: IterableDataset = load_dataset(
    "haydn-jones/ZINC20", split="test", streaming=True
)

samples = dataset.shuffle(seed=225).take(100)["smiles"]
original_smiles = list(samples)

Resolving data files:   0%|          | 0/787 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/96 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/93 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/787 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/96 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/93 [00:00<?, ?it/s]

In [3]:
from batchata import Batch

batch = (
    Batch(
        results_dir="./data/",
        max_parallel_batches=10,
        items_per_batch=10,
    )
    .set_default_params(model="gemini-2.5-flash")
    .add_cost_limit(10.0)
)

prompt_template = "Write the IUPAC name of this molecule:\n\n%s\n\nWrite the IUPAC name only. Do not write any comments."
for smiles in original_smiles:
    messages = [{"role": "user", "content": prompt_template % smiles}]
    batch.add_job(messages, max_tokens=8192)

In [4]:
# dry run the batch
run = batch.run(dry_run=True)

2026-01-28 09:52:24 - batchata.core.batch_run - INFO - Created temporary state file: /var/folders/5k/k24pr0ns75zcrcwfppb15tph0000gn/T/tmpokhh5l5h.json
2026-01-28 09:52:24 - batchata.core.batch_run - INFO - === DRY RUN MODE ===
2026-01-28 09:52:24 - batchata.core.batch_run - INFO - This will show cost estimates without executing jobs
2026-01-28 09:52:24 - batchata.core.batch_run - INFO - Analyzing 100 pending jobs...
2026-01-28 09:52:24 - batchata.core.batch_run - INFO - 
Job breakdown:
2026-01-28 09:52:24 - batchata.core.batch_run - INFO - 
GeminiProvider (100 jobs):
2026-01-28 09:52:25 - batchata.core.batch_run - INFO -   Batch 1: 10 jobs, estimated cost: $0.1025
2026-01-28 09:52:25 - batchata.core.batch_run - INFO -     - job-052eceb3: direct messages (citations: False)
2026-01-28 09:52:25 - batchata.core.batch_run - INFO -     - job-fb08de35: direct messages (citations: False)
2026-01-28 09:52:25 - batchata.core.batch_run - INFO -     - job-72892362: direct messages (citations: Fals

In [5]:
run = batch.run(print_status=True)

2026-01-28 09:52:31 - batchata.core.batch_run - INFO - Created temporary state file: /var/folders/5k/k24pr0ns75zcrcwfppb15tph0000gn/T/tmpyf276rwd.json
2026-01-28 09:52:31 - batchata.core.batch_run - INFO - Starting batch run


Output()

2026-01-28 09:52:38 - batchata.core.batch_run - INFO - Estimating cost for batch of 10 jobs...
2026-01-28 09:52:38 - batchata.core.batch_run - INFO - Estimating cost for batch of 10 jobs...
2026-01-28 09:52:38 - batchata.core.batch_run - INFO - Estimating cost for batch of 10 jobs...
2026-01-28 09:52:38 - batchata.core.batch_run - INFO - Estimating cost for batch of 10 jobs...
2026-01-28 09:52:38 - batchata.core.batch_run - INFO - Estimating cost for batch of 10 jobs...
2026-01-28 09:52:38 - batchata.core.batch_run - INFO - Estimating cost for batch of 10 jobs...
2026-01-28 09:52:38 - batchata.core.batch_run - INFO - Estimating cost for batch of 10 jobs...
2026-01-28 09:52:38 - batchata.core.batch_run - INFO - Estimating cost for batch of 10 jobs...
2026-01-28 09:52:38 - batchata.core.batch_run - INFO - Estimating cost for batch of 10 jobs...
2026-01-28 09:52:38 - batchata.core.batch_run - INFO - Estimating cost for batch of 10 jobs...
2026-01-28 09:52:39 - batchata.core.batch_run - IN

2026-01-28 10:00:05 - batchata.core.batch_run - INFO - Getting results for batch batches/j54xaw0urgq7pcao7l3m1w2hg1hndmpo6t40
2026-01-28 10:00:05 - batchata.providers.provider - WARNING - Failed to save raw responses for batch batches/j54xaw0urgq7pcao7l3m1w2hg1hndmpo6t40: Object of type GenerateContentResponse is not JSON serializable
2026-01-28 10:00:05 - batchata.core.batch_run - INFO - ✓ Batch batches/j54xaw0urgq7pcao7l3m1w2hg1hndmpo6t40 completed: 10 success, 0 failed, cost: $0.000567
2026-01-28 10:00:05 - batchata.core.batch_run - INFO - ✓ Job job-318220bc completed successfully
2026-01-28 10:00:05 - batchata.core.batch_run - INFO - ✓ Job job-91d7f343 completed successfully
2026-01-28 10:00:05 - batchata.core.batch_run - INFO - ✓ Job job-8aa8da6a completed successfully
2026-01-28 10:00:05 - batchata.core.batch_run - INFO - ✓ Job job-3f1809f4 completed successfully
2026-01-28 10:00:05 - batchata.core.batch_run - INFO - ✓ Job job-203c374b completed successfully
2026-01-28 10:00:05 -

2026-01-28 10:00:07 - batchata.core.batch_run - INFO - Getting results for batch batches/zk1kcf9uyph4ipyugfy9zasrunokkoltgwuo
2026-01-28 10:00:07 - batchata.providers.provider - WARNING - Failed to save raw responses for batch batches/zk1kcf9uyph4ipyugfy9zasrunokkoltgwuo: Object of type GenerateContentResponse is not JSON serializable
2026-01-28 10:00:07 - batchata.core.batch_run - INFO - ✓ Batch batches/zk1kcf9uyph4ipyugfy9zasrunokkoltgwuo completed: 10 success, 0 failed, cost: $0.000616
2026-01-28 10:00:07 - batchata.core.batch_run - INFO - ✓ Job job-c5e29970 completed successfully
2026-01-28 10:00:07 - batchata.core.batch_run - INFO - ✓ Job job-20e3b693 completed successfully
2026-01-28 10:00:07 - batchata.core.batch_run - INFO - ✓ Job job-1b58cff8 completed successfully
2026-01-28 10:00:07 - batchata.core.batch_run - INFO - ✓ Job job-79d648a6 completed successfully
2026-01-28 10:00:07 - batchata.core.batch_run - INFO - ✓ Job job-060718bc completed successfully
2026-01-28 10:00:07 -

2026-01-28 10:00:08 - batchata.core.batch_run - INFO - Getting results for batch batches/v56zvj9fode4ivdrqudall3rna73rnv9ytk6
2026-01-28 10:00:08 - batchata.providers.provider - WARNING - Failed to save raw responses for batch batches/v56zvj9fode4ivdrqudall3rna73rnv9ytk6: Object of type GenerateContentResponse is not JSON serializable
2026-01-28 10:00:08 - batchata.core.batch_run - INFO - ✓ Batch batches/v56zvj9fode4ivdrqudall3rna73rnv9ytk6 completed: 10 success, 0 failed, cost: $0.000601
2026-01-28 10:00:08 - batchata.core.batch_run - INFO - ✓ Job job-052eceb3 completed successfully
2026-01-28 10:00:08 - batchata.core.batch_run - INFO - ✓ Job job-fb08de35 completed successfully
2026-01-28 10:00:08 - batchata.core.batch_run - INFO - ✓ Job job-72892362 completed successfully
2026-01-28 10:00:08 - batchata.core.batch_run - INFO - ✓ Job job-3ed1faba completed successfully
2026-01-28 10:00:08 - batchata.core.batch_run - INFO - ✓ Job job-dc79a1a4 completed successfully
2026-01-28 10:00:08 -

2026-01-28 10:00:09 - batchata.core.batch_run - INFO - Getting results for batch batches/dkfrpf1ku38gw3jbdn8liv6zah3uw36gwpkf
2026-01-28 10:00:09 - batchata.providers.provider - WARNING - Failed to save raw responses for batch batches/dkfrpf1ku38gw3jbdn8liv6zah3uw36gwpkf: Object of type GenerateContentResponse is not JSON serializable
2026-01-28 10:00:09 - batchata.core.batch_run - INFO - ✓ Batch batches/dkfrpf1ku38gw3jbdn8liv6zah3uw36gwpkf completed: 10 success, 0 failed, cost: $0.000502
2026-01-28 10:00:09 - batchata.core.batch_run - INFO - ✓ Job job-5f26e4a2 completed successfully
2026-01-28 10:00:09 - batchata.core.batch_run - INFO - ✓ Job job-84d89df8 completed successfully
2026-01-28 10:00:09 - batchata.core.batch_run - INFO - ✓ Job job-fc2194fc completed successfully
2026-01-28 10:00:09 - batchata.core.batch_run - INFO - ✓ Job job-676a4162 completed successfully
2026-01-28 10:00:09 - batchata.core.batch_run - INFO - ✓ Job job-09ae6afb completed successfully
2026-01-28 10:00:09 -

2026-01-28 10:00:11 - batchata.core.batch_run - INFO - Getting results for batch batches/n45woeu845jo69mcsiyaj8drak38dwo2z45f
2026-01-28 10:00:11 - batchata.providers.provider - WARNING - Failed to save raw responses for batch batches/n45woeu845jo69mcsiyaj8drak38dwo2z45f: Object of type GenerateContentResponse is not JSON serializable
2026-01-28 10:00:11 - batchata.core.batch_run - INFO - ✓ Batch batches/n45woeu845jo69mcsiyaj8drak38dwo2z45f completed: 10 success, 0 failed, cost: $0.000740
2026-01-28 10:00:11 - batchata.core.batch_run - INFO - ✓ Job job-2b628c22 completed successfully
2026-01-28 10:00:11 - batchata.core.batch_run - INFO - ✓ Job job-067073e1 completed successfully
2026-01-28 10:00:11 - batchata.core.batch_run - INFO - ✓ Job job-7c31d31f completed successfully
2026-01-28 10:00:11 - batchata.core.batch_run - INFO - ✓ Job job-2ca7bded completed successfully
2026-01-28 10:00:11 - batchata.core.batch_run - INFO - ✓ Job job-0173f938 completed successfully
2026-01-28 10:00:11 -

In [6]:
results = run.results()
for k, v in results.items():
    print(k, len(v))

completed 100
failed 0
cancelled 0


In [8]:
# convert warnings to exceptions
import warnings

warnings.filterwarnings("error")

# create temp path for opsin input
import tempfile
import pathlib

tmp_opsin_path = str(pathlib.Path(tempfile.gettempdir()) / "py2opsin_input.txt")

from py2opsin import py2opsin

completed = results["completed"]
iupacs = []
smiles = []
errors = []
for job in completed:
    response = job.raw_response
    try:
        smi = py2opsin(response, "SMILES", tmp_fpath=tmp_opsin_path)
        err = ""
    except RuntimeWarning as e:
        smi = ""
        err = (
            str(e)
            .replace("\n", " ")
            .removeprefix("OPSIN raised the following error(s) while parsing:  > ")
        )
    iupacs.append(response)
    errors.append(err)
    smiles.append(smi)

In [9]:
import polars as pl

df = pl.DataFrame(
    {
        "ZINC SMILES": original_smiles,
        "Generated IUPAC": iupacs,
        "OPSIN SMILES": smiles,
        "OPSIN Errors": errors,
    }
)
df.write_csv("data-collected.csv")